# Assignment 2A

**Name**: SMRITI TRIPATHI                              </br>
**Roll No.**: 200988 

In [1]:
# import
import gymnasium as gym
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import random

In [2]:
'''
Represents a Stochastic Maze problem Gym Environment which provides a Fully observable
MDP
'''
class StochasticMazeEnv(gym.Env):
    '''
    StochasticMazeEnv represents the Gym Environment for the Stochastic Maze environment
    States : [0,1,2,3,4,5,6,7,8,9,10,11]
    Actions : ["Left":0, "Up":1, "Right":2, "Down":3]
    '''
    metadata = {'render.modes': ['human']}

    def __init__(self,initial_state=0,no_states=12,no_actions=4):
        '''
        Constructor for the StochasticMazeEnv class

        Args:
            initial_state : starting state of the agent
            no_states : The no. of possible states which is 12
            no_actions : The no. of possible actions which is 4
            
        '''
        self.initial_state = initial_state
        self.state = self.initial_state
        self.nA = no_actions
        self.nS = no_states
        self.actions_dict = {"L":0, "U":1, "R":2, "D":3}
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        self.reset()

    def reset(self):
        '''
        Resets the environment
        Returns:
            observations containing player's current state
        '''
        self.state = self.initial_state
        return self.get_obs()

    def get_obs(self):
        '''
        Returns the player's state as the observation of the environment
        '''
        return (self.state)

    def render(self, mode='human'):
        '''
        Renders the environment
        '''
        print("Current state: {}".format(self.state))

    def sample_action(self):
        '''
        Samples and returns a random action from the action space
        '''
        return random.randint(0, self.nA)
    def P(self):
        '''
        Defines and returns the probabilty transition matrix which is in the form of a nested dictionary
        '''
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        return self.prob_dynamics


    def step(self, action):
        '''
        Performs the given action
        Args:
            action : action from the action_space to be taking in the environment
        Returns:
            observation - returns current state
            reward - reward obtained after taking the given action
            done - True if the episode is complete else False
        '''
        if action >= self.nA:
            action = self.nA-1

        index = np.random.choice(3,1,p=[0.8,0.1,0.1])[0]

        dynamics_tuple = self.prob_dynamics[self.state][action][index]
        self.state = dynamics_tuple[1]
        

        return self.state, dynamics_tuple[2], dynamics_tuple[3]

In [3]:
env = StochasticMazeEnv()
env.reset()
num_states = env.nS
num_actions = env.nA

## Test Cases for checking the environment 

## Example: Random Policy

In [4]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1

    rand_action = np.random.choice(4,1)[0]  #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(rand_action)
    
    total_reward += reward

    print("  ", rand_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   0 	   0 	 -0.01 	   False
   3 	   4 	 -0.01 	   False
   0 	   4 	 -0.01 	   False
   3 	   4 	 -0.01 	   False
   3 	   8 	 -0.01 	   False
   1 	   4 	 -0.01 	   False
   1 	   0 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   1 	   0 	 -0.01 	   False
   3 	   4 	 -0.01 	   False
   1 	   0 	 -0.01 	   False
   3 	   4 	 -0.01 	   False
   1 	   0 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   3 	   4 	 -0.01 	   False
   3 	   8 	 -0.01 	   False
   0 	   8 	 -0.01 	   False
   0 	   8 	 -0.01 	   False
   2 	   9 	 -0.01 	   False
   3 	   9 	 -0.01 	   False
   3 	   9 	 -0.01 	   False
   3 	   9 	 -0.01 	   False
   0 	   8 	 -0.01 	   False
   2 	   4 	 -0.01 	   False
   3 	   8 	 -0.01 	   False
   1 	   4 	 -0.01 	   False
   1 	   0 	 -0.01 	   False
   3 	   1 	 -0.01 	   False
   1 	   2 	 -0.01 	   False
   3 	   6 	 -0.01 	   False
   2 	   7 	 -1 	   True
Total Number

### The random policy takes large number of steps to reach some terminal state which should be much higher than the number of the steps taken by a all 'Right' policy

## Example: Right Policy

In [5]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1

    right_action = 2  #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(right_action)
    
    total_reward += reward

    print("  ", right_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   2 	   3 	 1 	   True
Total Number of steps to Reach Terminal: 3
Final Reward: 0.98


### The right policy most of the time reaches the goal state in just 3 steps which is expected

***

## Q1. Find an optimal policy to navigate the given environment using Policy Iteration 

In [8]:
# write your code here
# hint: code policy evaluation function
# hint: code policy improvement function
# hint: implement policy iteration

# let us assume gamma as 1
# We'll compute V - 10000 times which can be approximated as calculating V infinite times as done in Policy Iteration
# K= 10000

class Policy_Iteration:
    def __init__(self, env):
        self.env = env
        self.policy = np.ones([env.nS, env.nA]) / env.nA 
        self.values = np.zeros(env.nS)

    def policy_evaluation(self):
        for i in range(0,10000):
            for state in range(self.env.nS):
                if state == 5:  # Skip the wall
                    continue
                v = self.values[state]
                v_new = 0
                for action, action_probability in enumerate(self.policy[state]):
                    iterations = self.env.P()[state][action]
                    for probability, next_state, reward, done in iterations:
                        v_new += action_probability * probability *(reward + 1 * self.values[next_state])

                self.values[state] = v_new

    def policy_improvement(self):
        policy_stable = True
        for state in range(self.env.nS):
            if state == 5:  # Skip the wall
                continue
            old_action = np.argmax(self.policy[state])
            action_values = np.zeros(self.env.nA)
            for action in range(self.env.nA):
                iterations = self.env.P()[state][action]
                for probability, next_state, reward, done in iterations:
                    action_values[action] += probability *(reward + 1 * self.values[next_state])
            best_action = np.argmax(action_values)

            if old_action != best_action:
                policy_stable = False

            self.policy[state] = np.eye(self.env.nA)[best_action] #being greedy

        return policy_stable

    def policy_iteration(self):
        while True:
            self.policy_evaluation()
            policy_stable = self.policy_improvement()
            if policy_stable:
                break


#Creating the environment
env = StochasticMazeEnv()

#Creating Policy Iteration agent
agent = Policy_Iteration(env)

#Performing Policy Iteration
agent.policy_iteration()

#Optimal Policy
optimal_policy = np.argmax(agent.policy, axis=1)

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")
state = env.reset()
total_reward = 0
count = 0
while True:
    rand_action = optimal_policy[state]
    next_state, reward, is_Terminal = env.step(rand_action) 
    print("  ", rand_action, "\t  ", next_state, "\t", reward, "\t  ", is_Terminal)
    total_reward += reward
    count += 1
    if is_Terminal:
        break
    state = next_state


print(f"Total Number of steps to Reach Terminal: {count}")
print(f"Final Reward: {total_reward}")

Action	 State	 Reward	 is_Terminal
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   2 	   3 	 1 	   True
Total Number of steps to Reach Terminal: 3
Final Reward: 0.98


## Q2. Find an optimal policy to navigate the given environment using Value Iteration

In [9]:
# write your code here
# let us assume gamma as 1
# We perform Policy Evaluation once in Value Iteration

class Value_Iteration:

    def __init__(self, env):
        self.env = env
        self.values = np.zeros(env.nS) 

    def value_iteration(self):
        for i in range(0,3):
            for s in range(self.env.nS):
                v = self.values[s]
                self.values[s] = max(self.q_pie(s, a) for a in range(self.env.nA))

    def q_pie(self, state, action):
        q = 0
        if state in self.env.prob_dynamics:
            for probability, next_state, reward, done in self.env.prob_dynamics[state][action]:
                q += probability * (reward + 1 * self.values[next_state])
        return q

    def extract_policy(self):
        policy = np.zeros(self.env.nS, dtype=int)
        for s in range(self.env.nS):
            q_values = [self.q_pie(s, a) for a in range(self.env.nA)]
            policy[s] = np.argmax(q_values)
        return policy


#Creating the environment
env = StochasticMazeEnv()

#Creating Value iteration agent
agent = Value_Iteration(env)

#Performing Value Iteration
agent.value_iteration()

#Optimal Policy
optimal_policy = agent.extract_policy()

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")
state = env.reset()
total_reward = 0
count = 0
while True:
    rand_action = optimal_policy[state]
    next_state, reward, is_Terminal = env.step(rand_action)
    count += 1
    total_reward += reward
    print("  ", rand_action, "\t  ", next_state, "\t", reward, "\t  ", is_Terminal)
    if is_Terminal:
        break
    state = next_state


print(f"Total Number of steps to Reach Terminal: {count}")
print(f"Final Reward: {total_reward}")


Action	 State	 Reward	 is_Terminal
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   2 	   3 	 1 	   True
Total Number of steps to Reach Terminal: 3
Final Reward: 0.98


## Q3. Compare PI and VI in terms of convergence. Is the policy obtained by both same?

### Answer: 
Both Value Iteration and Policy Iteration Algorithms result in the same policy. 
Over here we can see that both the policies converged in the same number of iterations but generally policy iteration converges faster than value iteration algorithm.